In [31]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# %pip install transformers[torch] datasets accelerate>=0.26.0
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [32]:
import random

# Generate train data

In [33]:
# import labels and relationships from the json files
import json
with open('./../../useful_dataset/graph/unique_movies.json') as f:
    file = json.load(f)
    entities = file

with open('./../../useful_dataset/graph/unique_relationships.json.') as f:
    file = json.load(f)
    relations = file

In [34]:
len(entities), len(relations)

(23065, 147)

In [35]:
templates = [
    "Who is the {} of {}",
    "Who {} the movie {}",
    "Who worked as {} on the movie {}",
    "Which person is the {} of {}"
]

In [36]:
def generate_examples(movies, relationships, templates, num_examples=100):
    sentences = []
    labels = []

    for _ in range(num_examples):
        # Randomly pick a relationship and a movie title
        relationship = random.choice(relationships)
        movie = random.choice(movies)

        # Randomly pick a template and fill it in
        template = random.choice(templates)
        question = template.format(relationship, movie)

        # Tokenize question manually
        tokens = question.split()

        # Create labels for each token
        label_seq = []
        for token in tokens:
            if token == relationship.split(" ")[0]:
                # label_seq.append("B-" + relationship.replace(" ", "_").upper())
                label_seq.append("B-RELATION")
            elif token in relationship.split(" ")[1:]:
                # label_seq.append("I-" + relationship.replace(" ", "_").upper())
                label_seq.append("I-RELATION")

            elif token == movie.split(" ")[0]:
                label_seq.append("B-MOVIE")
            elif token in movie.split()[1:]:
                label_seq.append("I-MOVIE")
            else:
                label_seq.append("O")

        # Append to the list
        sentences.append(tokens)
        labels.append(label_seq)

    return sentences, labels

# Generate 200 examples
tokens, ner_tags = generate_examples(entities, relations, templates, num_examples=5000)

In [37]:
id = 123
print(tokens[id])
print(ner_tags[id])

['Who', 'dedicated', 'to', 'the', 'movie', 'The', 'Sweet', 'and', 'the', 'Bitter']
['O', 'B-RELATION', 'I-RELATION', 'I-MOVIE', 'O', 'B-MOVIE', 'I-MOVIE', 'I-MOVIE', 'I-MOVIE', 'I-MOVIE']


In [38]:
# create a checkpoint of the created dataset with pickle?
import pickle

with open('data_for_ner.pkl', 'wb') as f:
    pickle.dump((tokens, ner_tags), f)


In [39]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import random
from transformers import DataCollatorForTokenClassification

# Load your dataset in the correct format
train_data = {
    'tokens': tokens,
    'ner_tags': ner_tags
}

# Map NER tags to labels (B-RELATION, B-MOVIE, etc.)
label_list = ["O", "B-RELATION", "B-MOVIE", "I-MOVIE", "I-RELATION"]

# Create the mapping from label to index
label_map = {label: i for i, label in enumerate(label_list)}

# Convert ner_tags to numerical values based on label_map
def convert_labels_to_ids(ner_tags):
    converted_tags = []
    for labels in ner_tags:
        converted_tags.append([label_map[label] for label in labels])
    return converted_tags

# Apply label conversion
train_data['ner_tags'] = convert_labels_to_ids(train_data['ner_tags'])

# Convert the dataset into Hugging Face Dataset format
dataset = Dataset.from_dict(train_data)

# Split the dataset into training and validation sets (80/20 split)
train_test_split = dataset.train_test_split(test_size=0.2)
datasets = DatasetDict({"train": train_test_split["train"], "test": train_test_split["test"]})

# Load a pre-trained NER-finetuned tokenizer and model
model_name = "elastic/distilbert-base-uncased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    ignore_mismatched_sizes=True  # Ignore mismatched sizes and initialize the classifier head for your label set
)

# Tokenize the dataset
# Function to tokenize and align labels
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples['tokens'], 
        truncation=True,  # Activate truncation
        # padding=True,     # Activate padding
        is_split_into_words=True  # Ensure input is treated as pre-tokenized (word level)
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word IDs for each token
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignored label for special tokens and padding
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Assign the label to the first token of a word
            else:
                label_ids.append(-100)  # Assign ignored label to sub-tokens
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization and label alignment to the dataset
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True, fn_kwargs={'tokenizer': tokenizer})

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at elastic/distilbert-base-uncased-finetuned-conll03-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1000/1000 [00:00<00:00, 20213.81 examples/s]


# Train Dataset

In [40]:
# Define training arguments
# Check if GPU is available and move model accordingly
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using device:", device)
model.to(device)

data_collator = DataCollatorForTokenClassification(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="no",
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
# Train the model
trainer.train()

# Save the model
model.save_pretrained("./custom_ner_model")
tokenizer.save_pretrained("./custom_ner_model")

Using device: cuda


('./custom_ner_model\\tokenizer_config.json',
 './custom_ner_model\\special_tokens_map.json',
 './custom_ner_model\\vocab.txt',
 './custom_ner_model\\added_tokens.json',
 './custom_ner_model\\tokenizer.json')

In [41]:
model.save_pretrained("./custom_ner_model")
tokenizer.save_pretrained("./custom_ner_model")

('./custom_ner_model\\tokenizer_config.json',
 './custom_ner_model\\special_tokens_map.json',
 './custom_ner_model\\vocab.txt',
 './custom_ner_model\\added_tokens.json',
 './custom_ner_model\\tokenizer.json')

# Test a sentence

In [42]:
import torch

# Assuming the model and tokenizer are already loaded into memory
# Remove loading code and pass the model and tokenizer as parameters

def predict_entities(input_text, model, tokenizer):
    # Define the label list (same as the training script)
    label_list = ["O", "B-RELATION", "B-MOVIE", "I-MOVIE", "I-RELATION"]
    
    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, is_split_into_words=False)
    
    # Move inputs to GPU if available
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print("Using device:", device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Run the model and get predictions
    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=2)
    
    # Convert token IDs to tokens and map predictions to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_labels = [label_list[pred.item()] for pred in predictions[0]]
    print(predicted_labels)

    # Format and display results
    output_text = ""
    for token, label in zip(tokens, predicted_labels):
        if token not in ["[CLS]", "[SEP]", "[PAD]"]:
            if label != "O":
                output_text += f"[{token} ({label})] "
            else:
                output_text += f"{token} "
    
    print(output_text)

predict_entities("Who is the director of star wars jedi - 4?", model, tokenizer)

Using device: cuda
['O', 'O', 'O', 'O', 'B-RELATION', 'O', 'B-MOVIE', 'I-MOVIE', 'I-MOVIE', 'I-MOVIE', 'I-MOVIE', 'I-MOVIE', 'O']
who is the [director (B-RELATION)] of [star (B-MOVIE)] [wars (I-MOVIE)] [jedi (I-MOVIE)] [- (I-MOVIE)] [4 (I-MOVIE)] [? (I-MOVIE)] 
